# Cuaderno de Entrenamiento: Model Training

Este cuaderno guía el proceso de entrenamiento de un modelo de Stable Diffusion utilizando las imágenes preprocesadas y aumentadas.

# 1 limpio posibles datasets de pruebas anteriores

In [ ]:
import os
import shutil

dataset_dir = "./stable_diffusion_model"
if os.path.exists(dataset_dir):
    shutil.rmtree(dataset_dir)
    print(f"Dataset en {dataset_dir} eliminado.")
else:
    print(f"No se encontró el directorio {dataset_dir}.")

del dataset


No se encontró el directorio ./stable_diffusion_model.


NameError: name 'dataset' is not defined

# 2 Instalamos librerías necesarias

In [1]:
! pip install torch==2.3.0  # Upgrade to this version
! pip install  datasets accelerate -U  # Reinstall with the new torch version
!pip install transformers==4.28.0 diffusers==0.16.1
! pip install peft bitsandbytes -q  # May require checking for compatibility
! pip install torchvision 0.18.0 -q  # Compatible version
! pip install xformers 0.0.26.post1 -q  # Compatible version


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 4.1 MB/s eta 0:00:00
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [2]:
! pip install  torchvision torchtext torchaudio -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.9 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.17.1+cu121
    Uninstalling torchvision-0.17.1+cu121:
      Successfully uninstalled torchvision-0.17.1+cu121
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.17.1
    Uninstalling torchtext-0.17.1:
      Successfully uninstalled torchtext-0.17.1
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.2.1+cu121
    Uninstalling torchaudio-2.2.1+cu121:
      Successfully uninstalled torchaudio-2.2.1+cu121


In [3]:
from datasets import load_from_disk, DatasetDict, Dataset, Features, Value, Image
from torchvision.transforms import ToTensor, Resize, Normalize, Compose




# 3 y 4 Cargamos y etiquetamos las imágenes y preprocesamos las imágenes

In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
from datasets import load_from_disk, DatasetDict, Dataset, Features, Value, Image
from torchvision.transforms import ToTensor, Resize, Normalize, Compose

# Ruta del dataset en Google Drive
dataset_path = '/content/drive/MyDrive/dataset/train/'


In [7]:
# Comprobar si el dataset ya existe
if os.path.exists(dataset_path):
    print("Dataset already exists. Loading the dataset...")
    dataset = DatasetDict({"train": load_from_disk(dataset_path)})
else:
    print("Dataset does not exist. Creating a new dataset...")

    def load_images_with_labels(data_dir):
        images = []
        labels = []
        label_map = {
            "cuadros-coleccion": "cuadro de coleccion del barroco andino",
            "cuadros-figuras": "cuadro de figuras del barroco andino",
            "esculturas": "escultura del barroco andino",
            "esculturas-coleccion": "escultura de coleccion del barroco andino",
            "murales": "mural del barroco andino"
        }

        for label, prompt in label_map.items():
            folder_path = os.path.join(data_dir, label)
            for img_file in os.listdir(folder_path):
                img_path = os.path.join(folder_path, img_file)
                if img_file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif', 'tiff')):
                    images.append(img_path)
                    labels.append(prompt)

        return images, labels

    data_dir = "/content/drive/MyDrive/ACADEMICO/IA/Colab Notebooks/Sprint3/TFB/data/augmented"
    image_paths, prompts = load_images_with_labels(data_dir)

    features = Features({"image": Image(), "text": Value("string")})

    dataset = DatasetDict({
        "train": Dataset.from_dict({
            "image": image_paths,
            "text": prompts
        }, features=features)
    })

    # Preprocesar las Imágenes
    preprocess = Compose([
        Resize((512, 512)),
        ToTensor(),
        Normalize([0.5], [0.5]),
    ])

    def transform(example):
        example["pixel_values"] = preprocess(example["image"].convert("RGB"))
        return example

    dataset["train"] = dataset["train"].map(transform, batched=False)

    if "image" in dataset["train"].column_names:
        dataset["train"] = dataset["train"].remove_columns("image")

    # Guardar el dataset en Google Drive
    dataset["train"].save_to_disk(dataset_path)

# Verificar el número de ejemplos en el dataset después de cargar o crear
print(f"Number of examples in the train dataset: {len(dataset['train'])}")
print(f"Example: {dataset['train'][0]}")


Dataset already exists. Loading the dataset...


Loading dataset from disk:   0%|          | 0/32 [00:00<?, ?it/s]

Number of examples in the train dataset: 5000


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)





# 7 Configurar y Entrenar el Modelo

In [8]:
from transformers import CLIPTextModel, CLIPTokenizer, TrainingArguments, Trainer
from diffusers import UNet2DConditionModel, AutoencoderKL, DDPMScheduler
import torch
import json

# Configurar el Tokenizer y el Modelo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae").to(device)
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet").to(device)
noise_scheduler = DDPMScheduler.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="scheduler")

# Crear una copia mutable de la configuración del modelo
mutable_config = dict(unet.config)

# Crear una clase Trainer personalizada para evitar el problema con FrozenDict
class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.mutable_config = mutable_config
        # Añadir el método to_json_string a la configuración mutable
        self.mutable_config['to_json_string'] = lambda: json.dumps(self.mutable_config)

    def on_train_begin(self, args, state, control, **kwargs):
        # Reemplazar el método `to_json_string` en el modelo con el método de la configuración mutable
        if hasattr(self.model, 'config'):
            self.model.config.to_json_string = self.mutable_config['to_json_string']
        super().on_train_begin(args, state, control, **kwargs)

    def compute_loss(self, model, inputs, return_outputs=False):
        # Mueve los tensores a la GPU
        pixel_values = inputs["pixel_values"].to(device)
        text_inputs = self.tokenizer(inputs["text"], padding="max_length", truncation=True, max_length=77, return_tensors="pt").input_ids.to(device)

        # Forward pass through text encoder and UNet
        encoder_hidden_states = text_encoder(text_inputs).last_hidden_state
        noise_pred = unet(pixel_values, encoder_hidden_states)["sample"]

        # Calculate loss
        target = noise_scheduler.add_noise(pixel_values)
        loss = ((noise_pred - target) ** 2).mean()

        return (loss, noise_pred) if return_outputs else loss

# Configurar los Argumentos del Entrenamiento
training_args = TrainingArguments(
    output_dir="./stable_diffusion_model",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=1000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = CustomTrainer(
    model=unet,
    args=training_args,
    train_dataset=dataset["train"],
    tokenizer=tokenizer,
)

# Entrenar el Modelo
trainer.train()

# Guardar el Modelo
unet.save_pretrained("./stable_diffusion_model/unet")
vae.save_pretrained("./stable_diffusion_model/vae")
tokenizer.save_pretrained("./stable_diffusion_model/tokenizer")
text_encoder.save_pretrained("./stable_diffusion_model/text_encoder")


AttributeError: module 'jax.random' has no attribute 'KeyArray'

In [ ]:
# Convertir la configuración del modelo a un diccionario regular
unet_config = dict(unet.config)

# 8 Configurar los Argumentos del Entrenamiento

In [ ]:
training_args = TrainingArguments(
    output_dir="./stable_diffusion_model",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=1000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=10,
)


# 9 Configurar trainer

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # Mueve los tensores a la GPU
        pixel_values = inputs["pixel_values"].to(device)
        text_inputs = tokenizer(inputs["text"], padding="max_length", truncation=True, max_length=77, return_tensors="pt").input_ids.to(device)

        # Forward pass through text encoder and UNet
        encoder_hidden_states = text_encoder(text_inputs).last_hidden_state
        noise_pred = unet(pixel_values, encoder_hidden_states)["sample"]

        # Calculate loss
        target = noise_scheduler.add_noise(pixel_values)
        loss = ((noise_pred - target) ** 2).mean()

        return (loss, noise_pred) if return_outputs else loss

trainer = CustomTrainer(
    model=unet,
    args=training_args,
    train_dataset=dataset["train"],
    tokenizer=tokenizer,
    data_collator=None,  # Asegúrate de que no se intente acceder a la configuración del modelo en los callbacks
)


# 10 Entrenar el modelo

In [ ]:
trainer.train()

AttributeError: 'FrozenDict' object has no attribute 'to_json_string'

# 11 Guardar modelo

In [ ]:
unet.save_pretrained("./stable_diffusion_model/unet")
vae.save_pretrained("./stable_diffusion_model/vae")
tokenizer.save_pretrained("./stable_diffusion_model/tokenizer")
text_encoder.save_pretrained("./stable_diffusion_model/text_encoder")


KeyboardInterrupt: 